In [1]:
import pythia8
from pyjet import cluster, DTYPE_PTEPM
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.colors import LinearSegmentedColormap
plt.style.use("dark_paper")

In [2]:
def prtStable(pid):
    if abs(pid) == 211: return True #pion
    if abs(pid) == 321: return True #K+-
    if abs(pid) == 11: return True #electron
    if abs(pid) == 13: return True #muon
    if abs(pid) == 2212: return True #proton
    #
    #if abs(pid) == 22: return True #gamma
    #if abs(pid) == 130: return True #K_l
    return False
def heavyFlavor(pid):
    if abs(pid) == 411: return True
    if abs(pid) == 421: return True
    if abs(pid) == 431: return True
    if abs(pid) == 4122: return True
    if abs(pid) == 511: return True
    if abs(pid) == 521: return True
    if abs(pid) == 531: return True
    if abs(pid) == 5122: return True
    return False
def getData(prt):
    data = [prt.index(),prt.name(),prt.id(),prt.status(),prt.mother1(),prt.mother2(),
            prt.daughter1(),prt.daughter2(),prt.e(),prt.px(),prt.py(),prt.pz(),prt.m(),prt.pT(),prt.eta(),prt.phi(),prt.theta()]
    keys = ["N","NAME","ID","STATUS","M1","M2","D1","D2","E", "px", "py", "pz" , "mass","pT","eta","phi","theta"]
    return {key:value for key,value in zip(keys,data)}
def getDataLite(prt):
    data = [prt.index(), prt.e(),prt.px(),prt.py(),prt.pz()]
    keys = ["N", "E", "px", "py", "pz"]
    return {key:value for key,value in zip(keys,data)}
def getInfo(prt):
    string = "{:^2d} {:^11s} {:^5d} {:^6d} {:<3d} {:>3d}  {:<3d} {:>3d}    ".format(prt.index(),prt.name(),prt.id(),
                                                                       prt.status(),
                                                                       prt.mother1(),prt.mother2(),
                                                                       prt.daughter1(),prt.daughter2())
    string += (5*" {:>8.3f} ").format(prt.e(),prt.px(),prt.py(),prt.pz(),prt.m())
    return string

In [3]:
# Setup Pythia Parameters
# Generator. Shorthand for event.
pythia = pythia8.Pythia()

# Set up incoming beams, for frame with equal beam energies.
# BeamA = proton.
pythia.readString("Beams:idA = 2212")
pythia.readString("Beams:idB = 2212")
pythia.settings.parm("Beams:eCM", 14000.)
# QCD pp
pythia.readString("HardQCD:all = on");
pythia.readString("PhaseSpace:pTHatMin = 200.")

# Process selection.
# pythia.readString("WeakSingleBoson:ffbar2W = on");
# Switch off all W decays and then switch back on those to quarks.
# pythia.readString("24:onMode = off");
# pythia.readString("24:onIfAny = 1 2 3 4 5");

# Initialize.
print("Pythia generator initialized successfully: ", pythia.init())

Pythia generator initialized successfully:  True


In [4]:
def selection(prt):
    sel = prt.isFinal() and prt.vProd().pAbs()<1000 #and prtStable(prt.id()) # and prt.isCharged()
    return sel

In [5]:
# # Full Verbose for Debugging
# nEvents = 1
# events = list()
# for iEvent in range(1,nEvents+1):
#     if not pythia.next(): continue
#     event = pythia.event 
#     print("=="*20 + f" Event {iEvent:^3d}/{nEvents:^3d} " + "=="*20)
#     print(("{:^2s} {:^11s} {:^5s} {:^6s} {:^8s} {:^8s}   " + 5*"{:^10s}").format("N","NAME","ID","STATUS","PARENTS","CHILDREN","E", "px", "py", "pz" , "m"))
#     df = pd.DataFrame(columns=["N","NAME","ID","STATUS","M1","M2","D1","D2","E", "px", "py", "pz" , "mass","eta","phi","theta"])
#     for prt in event:
#         print(getInfo(prt))
#         if selection(prt):
#             df = df.append(getData(prt),ignore_index=True)
#         if abs(prt.id())==24: 
#             index = prt.index()
#             print("-"*100)
#     W = event[index] #get outgoing parton
#     df = df.append(getData(W),ignore_index=True)
#     df.name = f"Event {iEvent}"
#     #get outgoing parton daughters
#     fs_daughters = np.unique([daughterid for daughterid in W.daughterListRecursive()\
#                     if selection(event[daughterid])])
#     mask = np.zeros(len(df)).astype("int")
#     mask[df.N.isin(fs_daughters).tolist()] = 1
#     df["mask"] = mask
#     events.append(df)

In [ ]:
#%%capture out
nEvents = int(1e4)
for iEvent in range(0,nEvents):
    if not pythia.next(): continue
    event = pythia.event 
    print("=="*20 + f" Event {iEvent:^3d}/{nEvents:^3d} " + "=="*20,end="\r")
    df = pd.DataFrame(columns=["N", "E", "px", "py", "pz"])
    for prt in event:
        if selection(prt):#
            df = df.append(getDataLite(prt),ignore_index=True)
    df.loc[:,["E", "px", "py", "pz"]].to_hdf("data/QCD/evts_qcd.h5", key=f"evt{iEvent}")